### Evaluate Sartoris' detection method, creating Metrics and Points Per Scenario (runtime: ~ 30sec)

In [6]:
import matplotlib.pyplot as plt
import os
import numpy as np
import json

#### Set paths
<div class="alert alert-block alert-warning">
<h4>ToDo:</h4>
<ol>
    <li> Set "coda_root" to the root of your <b>CODA directory</b>
    <li> Set "inference_root" to the root of your <b>inference directory</b>.
    <li> Set "print_latex" to <b>True</b> for a <b>LateX-table</b> output, <b>False</b> for <b>visual</b> output
    <li> Set "plt_show" to <b>True</b> to show the <b>metrics-plots</b>, <b>False</b> to not show the <b>metrics-plots</b>. (This can slow down the environment, especially over VPN, since the plots add up to a lot of data.)
</ol>
</div>

In [7]:
coda_root = f'/disk/ml/own_datasets/CODA'
inference_root = f'/disk/vanishing_data/ju878/inference/'

print_latex = True
plt_show = False

In [8]:
model_root = os.path.realpath('../../model_contradictions/')
graphics_root = os.path.join(model_root, 'graphics')

eval_all_root = os.path.join(inference_root, 'groundtruth_eval_all')
eval_all_anno_root = os.path.join(inference_root, 'groundtruth_eval_all_annotation')
eval_all_counts_file = os.path.join(eval_all_root, 'counts.txt')
eval_all_anno_counts_file = os.path.join(eval_all_anno_root, 'counts.txt')

json_cornercases = os.path.join(coda_root, 'corner_case.json')

#### Prepare data for each dataset

Corresponding scenario to each position in the "eval_all_counts" array

    number in array:   0      1     2    3     4    5    6    7     8    9   10  11  12  13  14  15  16  17    18         19           20
    scenario:         c_11, c_12, c_13, c_14, c11, c12, c13, c14, c_10, c10, c1, c2, c3, c4, tp, fp, fn, tn, total, original_total, new_label_total

Load data for each dataset

In [9]:
# loads "counts.txt" from image based comparison between ground truth and detection method
eval_all_counts = np.loadtxt(eval_all_counts_file, delimiter=',')

# loads "counts.txt" from annotation based comparison between ground truth and detection method
eval_all_anno_counts = np.loadtxt(eval_all_anno_counts_file, delimiter=',')

eval_once_counts = eval_all_counts[:1034]
eval_kitti_counts = eval_all_counts[1034:1341]
eval_nuscenes_counts = eval_all_counts[1341:]

eval_once_anno_counts = eval_all_anno_counts[:4413]
eval_kitti_anno_counts = eval_all_anno_counts[4413:4812]
eval_nuscenes_anno_counts = eval_all_anno_counts[4812:]

# removes all 0 lines
eval_all_anno_counts_cut = eval_all_anno_counts[np.sum(eval_all_anno_counts, axis=1) != 0]
eval_once_anno_counts_cut = eval_once_anno_counts[np.sum(eval_once_anno_counts, axis=1) != 0]
eval_kitti_anno_counts_cut = eval_kitti_anno_counts[np.sum(eval_kitti_anno_counts, axis=1) != 0]
eval_nuscenes_anno_counts_cut = eval_nuscenes_anno_counts[np.sum(eval_nuscenes_anno_counts, axis=1) != 0]

eval_all = [eval_all_counts, eval_once_counts, eval_kitti_counts, eval_nuscenes_counts, 
            eval_all_anno_counts_cut, eval_once_anno_counts_cut, eval_kitti_anno_counts_cut, eval_nuscenes_anno_counts_cut]

with open(json_cornercases, 'r') as f:
    data_cornercases = json.load(f)
    
categories = data_cornercases['categories']
images = data_cornercases['images']
annotations = data_cornercases['annotations']

# removes all lines from "annotations", which were 0 in the "eval_***_anno_counts"
annotations_cut = []
for annotation in annotations:
    if not np.sum(eval_all_anno_counts[annotation['id'] - 1], axis=0) == 0:
        annotations_cut.append(annotation)

Create all labels

In [10]:
dataset_names = ['CODA', 'CODA-ONCE', 'CODA-KITTI', 'CODA-nuScenes', 'CODA-Anno', 'CODA-ONCE-Anno', 'CODA-KITTI-Anno', 'CODA-nuScenes-Anno']
categorie_names = ['sc_pedestrian', 'sc_cyclist', 'sc_vehicle', 'sc_animal', 'sc_traffic_facility', 'sc_obstruction', 'sc_misc', 
                   'c_pedestrian', 'c_cyclist', 'c_car', 'c_truck', 'c_tram', 'c_tricycle', 'c_bus', 'c_bicycle', 'c_moped', 'c_motorcycle',
                   'c_stroller', 'c_wheelchair', 'c_cart', 'c_trailer', 'c_construction_vehicle', 'c_recreational_vehicle', 'c_dog', 'c_barrier', 
                   'c_bollard', 'c_warning_sign', 'c_sentry_box', 'c_traffic_box', 'c_traffic_cone', 'c_traffic_island', 'c_traffic_light', 
                   'c_traffic_sign', 'c_debris', 'c_suitcace', 'c_dustbin', 'c_concrete_block', 'c_machinery', 'c_chair', 'c_phone_booth', 'c_basket',
                   'c_misc']
cm_names = ['All', 'Boxes', 'Overlap']
cm_anno_names = ['Boxes', 'Overlap']

labels = ['tp', 'fp', 'fn', 'tn']
title = ['All', 'Boxes', 'Overlap', 'Boxes_anno', 'Overlap_anno']

Divide data into supercategories and categories

In [11]:
supercategories = ['pedestrian', 'cyclist', 'vehicle', 'animal', 'traffic_facility', 'obstruction', 'misc']
sc_pedestrian = []
sc_cyclist = []
sc_vehicle = []
sc_animal = []
sc_traffic_facility = []
sc_obstruction = []
sc_misc = []
c_pedestrian = []
c_cyclist = []
c_car = []
c_truck = []
c_tram = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
c_tricycle = []
c_bus = []
c_bicycle = []
c_moped = []
c_motorcycle = []
c_stroller = []
c_wheelchair = []
c_cart = []
c_trailer = []
c_construction_vehicle = []
c_recreational_vehicle = []
c_dog = []
c_barrier = []
c_bollard = []
c_warning_sign = []
c_sentry_box = []
c_traffic_box = []
c_traffic_cone = []
c_traffic_island = []
c_traffic_light = []
c_traffic_sign = []
c_debris = []
c_suitcace = []
c_dustbin = []
c_concrete_block = []
c_machinery = []
c_chair = []
c_phone_booth = []
c_basket = []
c_misc = []

for index_anno, annotation in enumerate(annotations_cut):
    for categorie in categories:
        if annotation['category_id'] == categorie['id']:
            categorie_name = categorie['name']
            if f'c_{categorie_name}' in globals() and isinstance(globals()[f'c_{categorie_name}'], list):
                    globals()[f'c_{categorie_name}'].append(eval_all_anno_counts_cut[index_anno])
            for index_sc, sc in enumerate(supercategories):
                if categorie['supercategory'] == sc:
                    if f'sc_{sc}' in globals() and isinstance(globals()[f'sc_{sc}'], list):
                        globals()[f'sc_{sc}'].append(eval_all_anno_counts_cut[index_anno])

eval_anomalies = [sc_pedestrian, sc_cyclist, sc_vehicle, sc_animal, sc_traffic_facility, sc_obstruction, sc_misc,
                  c_pedestrian, c_cyclist, c_car, c_truck, c_tram, c_tricycle, c_bus, c_bicycle, c_moped, c_motorcycle,
                  c_stroller, c_wheelchair, c_cart, c_trailer, c_construction_vehicle, c_recreational_vehicle, c_dog, c_barrier, 
                  c_bollard, c_warning_sign, c_sentry_box, c_traffic_box, c_traffic_cone, c_traffic_island, c_traffic_light, 
                  c_traffic_sign, c_debris, c_suitcace, c_dustbin, c_concrete_block, c_machinery, c_chair, c_phone_booth, c_basket,
                  c_misc
                  ] 

#### Functions to create and plot all Metrics

Plot graphs

In [12]:
def plotgraph(counts, dataset, part):
    # Create a pie plot
    fig = plt.figure(figsize=(6, 4), tight_layout=False)
    fig.suptitle(f'{dataset} - {part}')
    ax = fig.add_subplot(121)
    ax.pie(counts[0], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[1]}')
    
    ax = fig.add_subplot(122)
    ax.pie(counts[1], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[2]}')
    
    fig.savefig(os.path.join(graphics_root, f'evaluate_detection_method/{dataset}_{part}.svg'), format='svg', dpi=1200)
    
    # Display the plot
    if plt_show:
        plt.show()
    plt.close()

def plotgraph_dataset(counts, counts_anno, dataset, part):
    
    # Create a pie plot
    fig = plt.figure(figsize=(14, 4), tight_layout=False)
    fig.suptitle(f'{dataset} - {part}')
    ax = fig.add_subplot(151)
    ax.pie(counts[0], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[0]}')
    
    ax = fig.add_subplot(152)
    ax.pie(counts[1], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[1]}')
    
    ax = fig.add_subplot(153)
    ax.pie(counts[2], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[2]}')
    
    ax = fig.add_subplot(154)
    ax.pie(counts_anno[0], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[3]}')
    
    ax = fig.add_subplot(155)
    ax.pie(counts_anno[1], labels=labels, autopct='%1.1f%%')
    ax.set_title(f'{title[4]}')

    fig.savefig(os.path.join(graphics_root, f'evaluate_detection_method/{dataset}_{part}.svg'), format='svg', dpi=1200)
    # Display the plot
    if plt_show:
        plt.show()
    plt.close()

Create IoU

In [13]:
def create_individual_iou(cms):
    iou = []
    for i in range(len(cms[0])):
        if (cms[0][i] + cms[1][i] + cms[2][i]) == 0:
            iou.append(0)
            continue
        iou.append((cms[0][i]) / (cms[0][i] + cms[1][i] + cms[2][i]))
    return iou

def create_aggregated_iou(cm):
    if not (cm[0] + cm[1] + cm[2]) == 0:
        iou = cm[0] / (cm[0] + cm[1] + cm[2])
    else:
        iou = 0    
    return iou

Create confusion matrices (cm)

In [14]:
def create_individual_clustered_cm(eval):
    cm_all = [[],[],[],[]]
    cm_boxes = [[],[],[],[]]
    cm_overlap = [[],[],[],[]]
    for e in eval:
        if not (e[18]) == 0:
            cm_all[0].append(e[14] / e[18])
            cm_all[1].append(e[15] / e[18])
            cm_all[2].append(e[16] / e[18])
            cm_all[3].append(e[17] / e[18])

        if not (e[19]) == 0:
            cm_boxes[0].append((e[6] + e[7]) / e[19])
            cm_boxes[1].append((e[2] + e[3]) / e[19])
            cm_boxes[2].append((e[4] + e[5] + e[9]) / e[19])
            cm_boxes[3].append((e[0] + e[1] + e[8]) / e[19])

        if not (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]) == 0:
            cm_overlap[0].append((e[6] + e[7]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[1].append((e[2] + e[3]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[2].append((e[4] + e[5]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[3].append((e[0] + e[1]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
    
    return cm_all, cm_boxes, cm_overlap

def create_individual_cm(eval):
    cm_all = [[],[],[],[]]
    cm_boxes = [[],[],[],[]]
    cm_overlap = [[],[],[],[]]
    for e in eval:
        if not (e[18]) == 0:
            cm_all[0].append((e[2] + e[3] + e[6] + e[7]) / e[18])
            cm_all[1].append((e[12] + e[13]) / e[18])
            cm_all[2].append((e[0] + e[1] + e[4] + e[5] + e[8] + e[9]) / e[18])
            cm_all[3].append((e[10] + e[11]) / e[18])

        if not (e[19]) == 0:
            cm_boxes[0].append((e[2] + e[3] + e[6] + e[7]) / e[19])
            cm_boxes[1].append(0 / e[19])
            cm_boxes[2].append((e[0] + e[1] + e[4] + e[5] + e[8] + e[9]) / e[19])
            cm_boxes[3].append(0 / e[19])

        if not (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]) == 0:
            cm_overlap[0].append((e[2] + e[3] + e[6] + e[7]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[1].append(0 / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[2].append((e[0] + e[1] + e[4] + e[5] + e[8] + e[9]) / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
            cm_overlap[3].append(0 / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]))
    
    return cm_all, cm_boxes, cm_overlap

def create_aggregated_clustered_cm(eval):
    e = np.sum(eval, axis=0)
    cm_all = [e[14], e[15], e[16], e[17]]
    cm_boxes = [e[6] + e[7], e[2] + e[3], e[4] + e[5] + e[9], e[0] + e[1] + e[8]]
    cm_overlap = [e[6] + e[7], e[2] + e[3], e[4] + e[5], e[0] + e[1]]
    
        
    if not (e[18]) == 0:
        cm_all = cm_all / e[18]
    if not (e[19]) == 0:
        cm_boxes = cm_boxes / e[19]
    if not (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]) == 0:
        cm_overlap = cm_overlap / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7])
    
    return cm_all, cm_boxes, cm_overlap

def create_aggregated_cm(eval):
    e = np.sum(eval, axis=0)
    cm_all = [(e[2] + e[3] + e[6] + e[7]), (e[12] + e[13]), (e[0] + e[1] + e[4] + e[5] + e[8] + e[9]), (e[10] + e[11])]
    cm_boxes = [(e[2] + e[3] + e[6] + e[7]), 0, (e[0] + e[1] + e[4] + e[5] + e[8] + e[9]), 0]
    cm_overlap = [(e[2] + e[3] + e[6] + e[7]), 0, (e[0] + e[1] + e[4] + e[5] + e[8] + e[9]), 0]
    
        
    if not (e[18]) == 0:
        cm_all = cm_all / e[18]
    if not (e[19]) == 0:
        cm_boxes = cm_boxes / e[19]
    if not (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7]) == 0:
        cm_overlap = cm_overlap / (e[0] + e[1] + e[2] + e[3] + e[4] + e[5] + e[6] + e[7])
    
    return cm_all, cm_boxes, cm_overlap

Calculate all Metrics (individual and aggregated)

If "index" > 4 (eval_all) only boxes and overlap are calculated, since anomaly based data has only boxes and overlap

In [15]:
def get_individual(index, e, clustered):
    individual_count = []
    individual_iou = []
    individual_ap = []
    individual_ar = []
    individual_f1 = []
    individual_cm = []
    if clustered:
        confusion_matrices = create_individual_clustered_cm(e)
    else:
        confusion_matrices = create_individual_cm(e)
    for i, cms in enumerate(confusion_matrices):
        
        if index >= 4 and i == 0:
            continue
        
        # create iou
        m_iou = create_individual_iou(cms)
        if not len(m_iou) == 0:
            iou = sum(m_iou) / len(m_iou)
        else:
            iou = 0
        
        # create ap and ar
        precision = 0
        counter_precision = 0
        recall = 0
        counter_recall = 0
        for cm in range(len(cms[0])):
            counter_precision += 1
            counter_recall += 1
            if (cms[0][cm] + cms[1][cm]) == 0:
                continue
            precision += (cms[0][cm]) / (cms[0][cm] + cms[1][cm])
            if (cms[0][cm] + cms[2][cm]) == 0:
                continue
            recall += (cms[0][cm]) / (cms[0][cm] + cms[2][cm])
            
        if not counter_precision == 0:
            precision = (precision / counter_precision)
        if not counter_recall == 0:
            recall = (recall / counter_recall)
        
        # create f1
        if not (precision + recall) == 0:
            f1 = (2 * (precision * recall)) / (precision + recall)
        else:
            f1 = 0
        
        individual_count.append(counter_precision)
        individual_iou.append(iou)  
        individual_ap.append(precision)
        individual_ar.append(recall)
        individual_f1.append(f1)
        conf_mat = [0, 0, 0, 0]
        for cm in range(len(cms[0])):
            conf_mat[0] += cms[0][cm]
            conf_mat[1] += cms[1][cm]
            conf_mat[2] += cms[2][cm]
            conf_mat[3] += cms[3][cm]
        for c in range(len(conf_mat)):
            if not len(cms[0]) == 0:
                conf_mat[c] = conf_mat[c] / len(cms[0])
        individual_cm.append(conf_mat)
    
    return individual_iou, individual_ap, individual_ar, individual_f1, individual_cm, individual_count

In [16]:
def get_aggregated(index, e, clustered):
    aggregated_iou = []
    aggregated_ap = []
    aggregated_ar = []
    aggregated_f1 = []
    if clustered:
        confusion_matrices = create_aggregated_clustered_cm(e)
    else:
        confusion_matrices = create_aggregated_cm(e)
    for i, cms in enumerate(confusion_matrices):
        
        if index >= 4 and i == 0:
            continue
        # create iou
        iou = create_aggregated_iou(cms)
        
        # create ap and ar
        if not (cms[0] + cms[1]) == 0:
            precision = (cms[0]) / (cms[0] + cms[1])
        else:
            precision = 0
        if not (cms[0] + cms[2]) == 0:
            recall = (cms[0]) / (cms[0] + cms[2])
        else:
            recall = 0
        
        # create f1
        if not (precision + recall) == 0:
            f1 = (2 * (precision * recall)) / (precision + recall)
        else:
            f1 = 0
             
        aggregated_iou.append(iou)  
        aggregated_ap.append(precision)
        aggregated_ar.append(recall) 
        aggregated_f1.append(f1)
    
    if index >= 4:
        confusion_matrices = confusion_matrices[1:]
    return aggregated_iou, aggregated_ap, aggregated_ar, aggregated_f1, confusion_matrices

Print the outputs

In [17]:
def print_for_visual(index, i, individual_cl, aggregated_cl, individual, aggregated):
    
    print(f'Count: ' + str(individual[5][i]) + '\n')
                
    print('Clustered')
    print('individual IoU: ' + str(round(individual_cl[0][i] * 100, 1)))
    print('individual AP: ' + str(round(individual_cl[1][i] * 100, 1)))
    print('individual AR: ' + str(round(individual_cl[2][i] * 100, 1)))
    print('individual F1: ' + str(round(individual_cl[3][i] * 100, 1)))
    
    print('aggregated IoU: ' + str(round(aggregated_cl[0][i] * 100, 1)))
    print('aggregated AP: ' + str(round(aggregated_cl[1][i] * 100, 1)))
    print('aggregated AR: ' + str(round(aggregated_cl[2][i] * 100, 1)))
    print('aggregated F1: ' + str(round(aggregated_cl[3][i] * 100, 1)) + '\n')
    
    print('Not clustered')
    print('individual IoU: ' + str(round(individual[0][i] * 100, 1)))
    print('individual AP: ' + str(round(individual[1][i] * 100, 1)))
    print('individual AR: ' + str(round(individual[2][i] * 100, 1)))
    print('individual F1: ' + str(round(individual[3][i] * 100, 1)))
    
    print('aggregated IoU: ' + str(round(aggregated[0][i] * 100, 1)))
    print('aggregated AP: ' + str(round(aggregated[1][i] * 100, 1)))
    print('aggregated AR: ' + str(round(aggregated[2][i] * 100, 1)))
    print('aggregated F1: ' + str(round(aggregated[3][i] * 100, 1)) + '\n' + '\n')
    
def print_for_latex(i, name, individual_cl, aggregated_cl, individual, aggregated):
    print('& ' + name + ' & ' +
        str(individual[5][i]) + ' & ' +
        str(round(individual_cl[0][i] * 100, 1)) + ' & ' + 
        str(round(individual_cl[1][i] * 100, 1)) + ' & ' + 
        str(round(individual_cl[2][i] * 100, 1)) + ' & ' + 
        str(round(individual_cl[3][i] * 100, 1)) + ' & ' + 
        str(round(aggregated_cl[0][i] * 100, 1)) + ' & ' + 
        str(round(aggregated_cl[1][i] * 100, 1)) + ' & ' + 
        str(round(aggregated_cl[2][i] * 100, 1)) + ' & ' + 
        str(round(aggregated_cl[3][i] * 100, 1)) + ' & ' + 
        str(round(individual[0][i] * 100, 1)) + ' & ' + 
        str(round(individual[1][i] * 100, 1)) + ' & ' + 
        str(round(individual[2][i] * 100, 1)) + ' & ' + 
        str(round(individual[3][i] * 100, 1)) + ' & ' + 
        str(round(aggregated[0][i] * 100, 1)) + ' & ' + 
        str(round(aggregated[1][i] * 100, 1)) + ' & ' + 
        str(round(aggregated[2][i] * 100, 1)) + ' & ' + 
        str(round(aggregated[3][i] * 100, 1)) + ' \\\\'
        )

Main functions to combine all above

In [18]:
def get_IoU_AP_AR_F1_datasets(eval_all):
    all_individual = []
    all_aggregated = []
    for index, e in enumerate(eval_all):
        # create metrics for individual and aggregated metrics for each clustered and un-clustered ground truth
        individual_cl = get_individual(index, e, True)
        individual = get_individual(index, e, False)
        aggregated_cl = get_aggregated(index, e, True)
        aggregated = get_aggregated(index, e, False)
        
        # print outputs
        for i in range(len(individual[4])):
            print(f'{dataset_names[index]}: {cm_names[i]}')
            if print_latex:
                print_for_latex(i, dataset_names[index], individual_cl, aggregated_cl, individual, aggregated)
            else:
                print_for_visual(index, i, individual_cl, aggregated_cl, individual, aggregated)
        
        all_individual.append(individual[4])
        all_aggregated.append(aggregated[4])
    
    # create plots
    for index in range(4):
        plotgraph_dataset(all_individual[index], all_individual[index + 4], dataset_names[index], 'Individual')
        plotgraph_dataset(all_aggregated[index], all_aggregated[index + 4], dataset_names[index], 'Aggregated')
             

In [19]:
def get_IoU_AP_AR_F1_anomalies(eval_all):
    for index, e in enumerate(eval_all):
        # create metrics for individual and aggregated metrics for each clustered and un-clustered ground truth
        # the "index" for "get_individual" is set to 4 since all anomaly based data has only boxes and overlap
        individual_cl = get_individual(4, e, True)
        individual = get_individual(4, e, False)
        aggregated_cl = get_aggregated(4, e, True)
        aggregated = get_aggregated(4, e, False)
        
        # print outputs
        for i in range(len(individual[4])):
            print(f'{categorie_names[index]}: {cm_anno_names[i]}')
            if print_latex:
                print_for_latex(i, categorie_names[index].split('_')[1], individual_cl, aggregated_cl, individual, aggregated)
            else:
                print_for_visual(index, i, individual_cl, aggregated_cl, individual, aggregated)
        
        # create plots       
        plotgraph(individual[4], categorie_names[index], 'Individual')
        plotgraph(aggregated[4], categorie_names[index], 'Aggregated')

#### Create all metrics

In [20]:
get_IoU_AP_AR_F1_datasets(eval_all)

CODA: All
& CODA & 1412 & 8.9 & 13.2 & 26.2 & 17.5 & 10.8 & 13.7 & 33.5 & 19.5 & 9.2 & 16.2 & 21.2 & 18.4 & 11.8 & 16.8 & 28.3 & 21.1 \\
CODA: Boxes
& CODA & 1412 & 23.1 & 50.8 & 26.2 & 34.6 & 31.2 & 81.4 & 33.5 & 47.5 & 21.2 & 73.8 & 21.2 & 33.0 & 28.3 & 100.0 & 28.3 & 44.2 \\
CODA: Overlap
& CODA & 1304 & 32.4 & 55.0 & 37.9 & 44.9 & 40.5 & 81.4 & 44.6 & 57.6 & 23.0 & 79.9 & 23.0 & 35.7 & 28.3 & 100.0 & 28.3 & 44.2 \\
CODA-ONCE: All
& CODA-ONCE & 1034 & 8.9 & 14.0 & 27.1 & 18.4 & 8.8 & 11.1 & 29.4 & 16.1 & 9.0 & 16.9 & 21.5 & 18.9 & 9.9 & 14.2 & 24.5 & 18.0 \\
CODA-ONCE: Boxes
& CODA-ONCE & 1034 & 24.4 & 57.6 & 27.1 & 36.9 & 27.2 & 78.2 & 29.4 & 42.7 & 21.5 & 77.9 & 21.5 & 33.7 & 24.5 & 100.0 & 24.5 & 39.4 \\
CODA-ONCE: Overlap
& CODA-ONCE & 985 & 33.9 & 60.5 & 38.8 & 47.2 & 36.6 & 78.2 & 40.7 & 53.6 & 22.6 & 81.8 & 22.6 & 35.4 & 24.5 & 100.0 & 24.5 & 39.4 \\
CODA-KITTI: All
& CODA-KITTI & 307 & 10.9 & 13.3 & 29.0 & 18.3 & 14.9 & 19.0 & 40.9 & 25.9 & 11.8 & 17.4 & 25.0 & 20.6 & 16.0 &

In [21]:
get_IoU_AP_AR_F1_anomalies(eval_anomalies)

sc_pedestrian: Boxes
& pedestrian & 17 & 28.3 & 41.5 & 31.0 & 35.5 & 53.8 & 79.0 & 62.7 & 70.0 & 24.6 & 64.7 & 24.6 & 35.7 & 38.3 & 100.0 & 38.3 & 55.3 \\
sc_pedestrian: Overlap
& pedestrian & 16 & 33.9 & 44.1 & 37.3 & 40.4 & 57.7 & 79.0 & 68.1 & 73.2 & 26.2 & 68.8 & 26.2 & 37.9 & 38.3 & 100.0 & 38.3 & 55.3 \\
sc_cyclist: Boxes
& cyclist & 24 & 32.5 & 53.4 & 38.6 & 44.8 & 38.0 & 76.6 & 42.9 & 55.0 & 24.4 & 75.0 & 24.4 & 36.8 & 28.4 & 100.0 & 28.4 & 44.2 \\
sc_cyclist: Overlap
& cyclist & 22 & 41.6 & 58.3 & 49.5 & 53.5 & 49.0 & 76.6 & 57.6 & 65.8 & 26.6 & 81.8 & 26.6 & 40.1 & 28.4 & 100.0 & 28.4 & 44.2 \\
sc_vehicle: Boxes
& vehicle & 866 & 22.8 & 41.0 & 27.3 & 32.8 & 35.2 & 82.3 & 38.1 & 52.1 & 23.9 & 68.6 & 23.9 & 35.5 & 33.8 & 100.0 & 33.8 & 50.5 \\
sc_vehicle: Overlap
& vehicle & 736 & 33.0 & 48.3 & 41.0 & 44.4 & 43.3 & 82.3 & 47.7 & 60.4 & 28.2 & 80.7 & 28.2 & 41.8 & 33.8 & 100.0 & 33.8 & 50.5 \\
sc_animal: Boxes
& animal & 8 & 0.0 & 0.0 & 0.0 & 0 & 0.0 & 0 & 0.0 & 0 & 0.0 & 0.0 & 